#### import the things

In [1]:
import pandas as pd
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

idx = pd.IndexSlice

import scipy.stats

In [2]:
with open(r"..\path.txt") as p:
    path = p.readlines()[0]
import sys
sys.path.append(path)

import data_sorting_tools as dst

%load_ext autoreload
%autoreload 2

#### Understanding the key layout

In [3]:
raw_data_fol = r"..\data_files"

In [4]:
query1_file = os.path.join(raw_data_fol, "main_search1.json")
with open(query1_file) as f:
    query1 = json.load(f)

In [7]:
query1['0'].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'extended_tweet', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'possibly_sensitive', 'filter_level', 'lang', 'matching_rules', 'extended_text'])

#### create dataframe from main dataset

In [8]:
keep_keys = ['quote_count', 'reply_count', 'retweet_count', 'favorite_count', ('user','id'), 
                ('user','followers_count'), ('user','verified'), 'text', 'lang']

In [25]:
all_dfs = []
for f in os.listdir(raw_data_fol):
    if ("main" in f) and (".json" in f):
        d = dst.json_to_df(os.path.join(raw_data_fol, f), keep_keys, get_media=True, return_dict=False)
        all_dfs.append(d)
#         print(f, len(d))
        
df = pd.concat(all_dfs)

In [26]:
df.head()

,quote_count,reply_count,retweet_count,favorite_count,user_id,user_followers_count,user_verified,extended_text,text,lang,media_count,media_types
1362253193766338564,0,0,0,1,1281107755944652802,1053,False,The rippling effect of #SGBV by 🇪🇹 &amp; 🇪🇷 tr...,The rippling effect of #SGBV by 🇪🇹 &amp; 🇪🇷 tr...,en,0,none
1362062125443743745,7,7,354,827,4766873292,41473,False,NaN,#ForOurHopeWorld — Project 2\n\n@jhopeperu and...,en,0,none
1362253122215825410,0,0,0,2,916098322036903936,1725,False,@MarkDStrauss @RadioFreeTom Did he think about...,@MarkDStrauss @RadioFreeTom Did he think about...,en,0,none
1362182650623000579,0,0,2,5,20482638,129,False,NaN,@RhetoricPJ My dad’s car was filled with that ...,en,0,none
1361728845473140736,4,13,10,83,309728693,67308,True,NaN,Our COMPASS Initiative is focused on helping c...,en,0,none


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2889 entries, 1362253193766338564 to 1343490641230233601
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   quote_count           2889 non-null   object
 1   reply_count           2889 non-null   object
 2   retweet_count         2889 non-null   object
 3   favorite_count        2889 non-null   object
 4   user_id               2889 non-null   object
 5   user_followers_count  2889 non-null   object
 6   user_verified         2889 non-null   object
 7   extended_text         1292 non-null   object
 8   text                  2889 non-null   object
 9   lang                  2889 non-null   object
 10  media_count           2889 non-null   int64 
 11  media_types           2889 non-null   object
dtypes: int64(1), object(11)
memory usage: 293.4+ KB


In [28]:
text_compare = df[pd.isnull(df.extended_text)==False][['text', 'extended_text']]
text_compare

,text,extended_text
1362253193766338564,The rippling effect of #SGBV by 🇪🇹 &amp; 🇪🇷 tr...,The rippling effect of #SGBV by 🇪🇹 &amp; 🇪🇷 tr...
1362253122215825410,@MarkDStrauss @RadioFreeTom Did he think about...,@MarkDStrauss @RadioFreeTom Did he think about...
1362252698087608325,@joshgondelman So Kids becomes just a bunch of...,@joshgondelman So Kids becomes just a bunch of...
1362252108918104067,"@CyrilRamaphosa In the 50s they ""tried to"" cur...","@CyrilRamaphosa In the 50s they ""tried to"" cur..."
1362252063787245569,I'm discussing “Let’s talk about HIV &amp; PrE...,I'm discussing “Let’s talk about HIV &amp; PrE...
...,...,...
1344037226930438144,@JackPosobiec @MAGAreeni Spreading HIV on purp...,@JackPosobiec @MAGAreeni Spreading HIV on purp...
1344036889125212160,@ThomasUldrick @HPVAnalCancer .@ThomasUldrick ...,@ThomasUldrick @HPVAnalCancer .@ThomasUldrick ...
1344036814668103680,"Use condoms, HIV is real.\n\nUse condoms, unwa...","Use condoms, HIV is real.\n\nUse condoms, unwa..."
1344036806153670662,So you can knowingly give someone AIDS/HIV wit...,So you can knowingly give someone AIDS/HIV wit...


In [29]:
text_compare[text_compare.text == text_compare.extended_text]

,text,extended_text


In [30]:
text_compare.iloc[0].text

'The rippling effect of #SGBV by 🇪🇹 &amp; 🇪🇷 troops in Tigray will be devastating. The lack of medical facilities, rape… https://t.co/8NJfbyNtfB'

In [31]:
text_compare.iloc[0].extended_text

'The rippling effect of #SGBV by 🇪🇹 &amp; 🇪🇷 troops in Tigray will be devastating. The lack of medical facilities, rape kits and HIV medication means that people that urgently need medical attention will not be getting it. #TigrayGenocide  \n@UN_Women \nhttps://t.co/myLJcJiU3m'

In [32]:
df['extended_text'].fillna(df['text'], inplace=True)

In [33]:
df.drop('text', inplace=True, axis=1)
df.rename(columns={'extended_text':'text'}, inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2889 entries, 1362253193766338564 to 1343490641230233601
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   quote_count           2889 non-null   object
 1   reply_count           2889 non-null   object
 2   retweet_count         2889 non-null   object
 3   favorite_count        2889 non-null   object
 4   user_id               2889 non-null   object
 5   user_followers_count  2889 non-null   object
 6   user_verified         2889 non-null   object
 7   text                  2889 non-null   object
 8   lang                  2889 non-null   object
 9   media_count           2889 non-null   int64 
 10  media_types           2889 non-null   object
dtypes: int64(1), object(10)
memory usage: 270.8+ KB


In [35]:
df.set_index('user_id', drop=True, append=True, inplace=True)
df.index.rename(['tid', 'uid'], inplace=True)

In [36]:
df.head()

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types
tid,uid,,,,,,,,,,
1362253193766338564,1281107755944652802,0,0,0,1,1053,False,The rippling effect of #SGBV by 🇪🇹 &amp; 🇪🇷 tr...,en,0,none
1362062125443743745,4766873292,7,7,354,827,41473,False,#ForOurHopeWorld — Project 2\n\n@jhopeperu and...,en,0,none
1362253122215825410,916098322036903936,0,0,0,2,1725,False,@MarkDStrauss @RadioFreeTom Did he think about...,en,0,none
1362182650623000579,20482638,0,0,2,5,129,False,@RhetoricPJ My dad’s car was filled with that ...,en,0,none
1361728845473140736,309728693,4,13,10,83,67308,True,Our COMPASS Initiative is focused on helping c...,en,0,none


### duplicates

first, drop total duplicates

In [37]:
df[df.index.duplicated(keep=False)].sort_index()

quote_count reply_count retweet_count  \
tid                 uid                                                         
1329667044070068224 1731805261                  148          24          2213   
                    1731805261                  148          24          2213   
1337877651973955584 355696940                    39          13            49   
                    355696940                    39          13            49   
1341639291634642945 1284689967722180608         106          78           252   
                    1284689967722180608         106          78           252   
1343618572761919489 1382158386                    9          47           351   
                    1382158386                    9          47           351   
1348222998025809921 4910327794                    0           0             4   
                    4910327794                    0           0             4   
1351577913146032131 19177501                     10          38           638   
                    19177501                     10          38           638   
1353261234666287105 78621972                    126          43          2310   
                    78621972                    126          43          2310   
                    78621972                    126          43          2310   
1354372494191448066 4858377551                   18           2           100   
                    4858377551                   18           2           100   
1355960289825615874 19177501                      8          18           442   
                    19177501                      8          18           442   
1356267320717545472 1345516880                  247         204          2399   
                    1345516880                  247         204          2399   
1356286153369022471 20703607                    130          69           399   
                    20703607                    130          69           399   
1356321704713351168 354267800                     8           0            35   
                    354267800                     8           0            35   
1357817087515910144 78621972                     21           7           324   
                    78621972                     21           7           324   
1358792705728413696 19177501                      8          27           594   
                    19177501                      8          27           594   
1359230595293880320 19177501                      6          18           328   
                    19177501                      6          18           328   
1360193456673546243 453124512                     9          13            91   
                    453124512                     9          13            91   

                                        favorite_count user_followers_count  \
tid                 uid                                                       
1329667044070068224 1731805261                    6249                36366   
                    1731805261                    6249                36366   
1337877651973955584 355696940                       79               223554   
                    355696940                       79               223554   
1341639291634642945 1284689967722180608            991                 3882   
                    1284689967722180608            991                 3882   
1343618572761919489 1382158386                    3082               246518   
                    1382158386                    3082               246518   
1348222998025809921 4910327794                       7                 2612   
                    4910327794                       7                 2612   
1351577913146032131 19177501                     10049               281113   
                    19177501                     10049               281113   
1353261234666287105 78621972                      4873                41711   
                   

In [38]:
duplicates = df[df.index.duplicated(keep='first')].index
df.shape

(2889, 10)

In [39]:
df = df.drop(index=duplicates)
df.shape

(2856, 10)

In [40]:
df[df.duplicated(subset='text', keep=False)].sort_values('text')

,,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types
tid,uid,,,,,,,,,,
1352336727373967369,1141838062965665792,0,0,0,0,6154,False,. @LaurenJauregui please FOLLOW ME I have dep...,en,0,none
1352330769113489408,973551745371639808,0,4,1,9,8573,False,. @LaurenJauregui please FOLLOW ME I have dep...,en,0,none
1352324335525122066,1228037241496309760,1,1,0,0,3133,False,. @LaurenJauregui please FOLLOW ME I have dep...,en,0,none
1348309885759447041,1341227777400262657,1,0,4,8,22,False,NMAP thinks pilfering the 340B Discount from n...,en,0,none
1348310979566776320,1341275368578113536,0,0,6,6,19,False,NMAP thinks pilfering the 340B Discount from n...,en,0,none
1348310069121851394,1341227777400262657,1,0,3,8,22,False,NMAP’s vague proposal to loot the 340B Discoun...,en,0,none
1348311153978593282,1341275368578113536,0,0,1,1,19,False,NMAP’s vague proposal to loot the 340B Discoun...,en,0,none
1348310900839772160,1341275368578113536,1,0,2,3,19,False,NMAP’s “Right-of-Way” proposal puts nonprofit ...,en,0,none
1348309788552257536,1341227777400262657,0,0,3,4,22,False,NMAP’s “Right-of-Way” proposal puts nonprofit ...,en,0,none


In [41]:
more_duplicates = df[df.duplicated(subset='text', keep='first')].index
df.shape

(2856, 10)

In [42]:
df = df.drop(index=more_duplicates)
df.shape

(2843, 10)

check languages

In [43]:
df.lang.unique()

array(['en', 'tl', 'de', 'fr'], dtype=object)

In [44]:
df[df.lang!='en'].text.values

array(["Merry Christmas blood brothers. We're alive and kicking! And speaking of, shocks. Naalala ko.may hiv pala ako ung gamot ko! More than 4 hrs ng late !",
       '@ExtremNa @eberhati_ @Tobias52419476 @tagesschau * oder steckst andere mit HIV an',
       '#RT @pozmagazine: Our Solution @oriolgutierrez https://t.co/ZcVD6fTQZb #AfricanAmerican #coronavirus #COVID19',
       'Our Solution @oriolgutierrez https://t.co/8HQi55i26c #AfricanAmerican #coronavirus #COVID19'],
      dtype=object)

I feel comfortable dropping all of these tweets (I still really don't understand how they got through the tweepy language filter)

In [45]:
non_eng = df[df.lang!='en'].index
df.shape

(2843, 10)

In [46]:
df.drop(non_eng, inplace=True)
df.shape

(2839, 10)

In [47]:
df.groupby('tid').count().sort_values('quote_count', ascending=False)

,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,user_verified,text,lang,media_count,media_types
tid,,,,,,,,,,
511016028688486400,1,1,1,1,1,1,1,1,1,1
1356390372163903490,1,1,1,1,1,1,1,1,1,1
1356393547675668489,1,1,1,1,1,1,1,1,1,1
1356393556877987842,1,1,1,1,1,1,1,1,1,1
1356393813875429376,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
1348132587533959176,1,1,1,1,1,1,1,1,1,1
1348149061988339712,1,1,1,1,1,1,1,1,1,1
1348151588968427520,1,1,1,1,1,1,1,1,1,1


In [48]:
df = df.convert_dtypes()

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2839 entries, (1362253193766338564, 1281107755944652802) to (1343490641230233601, 2394619242)
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   quote_count           2839 non-null   Int64  
 1   reply_count           2839 non-null   Int64  
 2   retweet_count         2839 non-null   Int64  
 3   favorite_count        2839 non-null   Int64  
 4   user_followers_count  2839 non-null   Int64  
 5   user_verified         2839 non-null   boolean
 6   text                  2839 non-null   string 
 7   lang                  2839 non-null   string 
 8   media_count           2839 non-null   Int64  
 9   media_types           2839 non-null   string 
dtypes: Int64(6), boolean(1), string(3)
memory usage: 433.0 KB


In [50]:
df.describe()

,quote_count,reply_count,retweet_count,favorite_count,user_followers_count,media_count
count,2839.000000,2839.000000,2839.000000,2839.000000,2.839000e+03,2839.000000
mean,2.850652,4.155689,33.579782,162.766467,5.728812e+04,0.027827
std,23.648848,34.120156,289.821069,1590.076572,7.126107e+05,0.188464
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,0.000000,0.000000,0.000000,0.000000,2.360000e+02,0.000000
50%,0.000000,0.000000,0.000000,2.000000,1.187000e+03,0.000000
75%,0.000000,1.000000,5.000000,15.000000,6.139000e+03,0.000000
max,787.000000,1315.000000,8267.000000,57655.000000,2.577232e+07,4.000000


Since I was able to space the pulls further apart, there were far fewer repeats here, but it was also only a 5k pull. 

In [51]:
#save the dataframe as a pickle file to be loaded into other notebooks
pd.to_pickle(df, os.path.join(raw_data_fol, "full_dataset_cleaned_pull3.pkl"))